In [9]:
import os
import sys
import matplotlib.pyplot as plt
sys.path.insert(0, os.path.abspath('../'))
from market import market_wrapper
%matplotlib inline
sns.set_context(rc={'figure.figsize': (14, 7) } )
figzize_me = figsize =(14, 7)

btc_df = market_wrapper.create_k_data('BTC')
btc_df.head()
# standarlize data, index as close_date, all lower case

,key,open,high,low,close,adj close,volume,date_week,p_change
date,,,,,,,,,
2020-01-01,0,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997,2,NaN
2020-01-02,1,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465,3,-3.027293
2020-01-03,2,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032,4,5.017175
2020-01-04,3,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275,5,0.891501
2020-01-05,4,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095,6,0.008914


### 7.1.1 趋势跟踪和均值回复可能一个时间段内在一个币上都有所体现

In [10]:
import seaborn as sns
import numpy as np
import pandas as pd


sns.set_context(rc={'figure.figsize': (14, 7) } )
sns.regplot(x=np.arange(0, btc_df.shape[0]), y=btc_df.close.values, marker='+')
plt.show()

In [12]:
from utils import reg_util
deg = reg_util.calc_regress_deg(btc_df.close.values)
print('趋势角度:' + str(deg))

趋势角度:12.100046739107217


In [13]:
start = 0
# 前1/4的数据
end = int(btc_df.shape[0] / 4)
# 将x也使用arange切割
x = np.arange(start, end)
# y根据start，end进行切片
y = btc_df.close.values[start:end]
sns.regplot(x=x, y=y, marker='+')
plt.show()

In [14]:
start = int(btc_df.shape[0] / 4)
# 向前推1/4单位个时间
end = start + int(btc_df.shape[0] / 4)
sns.regplot(x=np.arange(start, end), y=btc_df.close.values[start:end],
            marker='+')
plt.show()

### 7.1.2 均值回复策略

In [33]:
max(btc_df.index)

Timestamp('2024-01-29 00:00:00')

In [20]:
# 头一年（[:252]）作为训练数据, 币圈交易中一年的交易日有365天
train_kl = btc_df[:365]
# 后一年（[252:]）作为回测数据
test_kl = btc_df[365:365*2]

# 分别画出两部分数据收盘价格曲线
tmp_df = pd.DataFrame(
    np.array([train_kl.close.values, test_kl.close.values]).T,
    columns=['train', 'test'])

tmp_df[['train', 'test']].plot(subplots=True, grid=True,
                               figsize=(14, 7));

In [21]:
# 训练数据的收盘价格均值
close_mean = train_kl.close.mean()
# 训练数据的收盘价格标准差
close_std = train_kl.close.std()

# 构造卖出信号阀值
sell_signal = close_mean + close_std / 3
# 构造买入信号阀值
buy_signal = close_mean - close_std / 3

# 可视化训练数据的卖出信号阀值，买入信号阀值及均值线
plt.figure(figsize=(14, 7))
# 训练集收盘价格可视化
train_kl.close.plot()
# 水平线，买入信号线, lw代表线的粗度
plt.axhline(buy_signal, color='r', lw=3)
# 水平线，均值线
plt.axhline(close_mean, color='black', lw=1)
# 水平线， 卖出信号线
plt.axhline(sell_signal, color='g', lw=3)
plt.legend(['train close', 'buy_signal', 'close_mean', 'sell_signal'],
           loc='best')
plt.show()

# 将卖出信号阀值，买入信号阀值代入回归测试数据可视化
plt.figure(figsize=(14, 7))
# 测试集收盘价格可视化
test_kl.close.plot()
# buy_signal直接代入买入信号
plt.axhline(buy_signal, color='r', lw=3)
# 直接代入训练集均值close
plt.axhline(close_mean, color='black', lw=1)
# sell_signal直接代入卖出信号
plt.axhline(sell_signal, color='g', lw=3)
# 按照上述绘制顺序标注
plt.legend(['test close', 'buy_signal', 'close_mean', 'sell_signal'],
           loc='best')
plt.show()

print('买入信号阀值:{} 卖出信号阀值:{}'.format(buy_signal, sell_signal))

买入信号阀值:9664.596496040158 卖出信号阀值:12470.157811536897


In [22]:
# 寻找测试数据中满足买入条件的时间序列
buy_index = test_kl[test_kl['close'] <= buy_signal].index

# 将找到的买入时间系列的信号设置为1，代表买入操作
test_kl.loc[buy_index, 'signal'] = 1
# 表7-2所示
test_kl[52:57]

,key,open,high,low,close,adj close,volume,date_week,p_change,signal
date,,,,,,,,,,
2021-02-21,417,56068.5664,58330.5703,55672.6094,57539.9453,57539.9453,51897585191,6,2.5352,NaN
2021-02-22,418,57532.7383,57533.3906,48967.5664,54207.3203,54207.3203,92052420332,0,-5.9663,NaN
2021-02-23,419,54204.9297,54204.9297,45290.5898,48824.4258,48824.4258,106102492824,1,-10.4585,NaN
2021-02-24,420,48835.0859,51290.1367,47213.5000,49705.3320,49705.3320,63695521388,2,1.7881,NaN
2021-02-25,421,49709.0820,51948.9688,47093.8516,47093.8516,47093.8516,54506565949,3,-5.3970,NaN


In [23]:
# 寻找测试数据中满足卖出条件的时间序列
sell_index = test_kl[test_kl['close'] >= sell_signal].index

# 将找到的卖出时间系列的信号设置为0，代表卖出操作
test_kl.loc[sell_index, 'signal'] = 0
# 表7-3所示
test_kl[48:53]

,key,open,high,low,close,adj close,volume,date_week,p_change,signal
date,,,,,,,,,,
2021-02-17,413,49207.2773,52533.9141,49072.3789,52149.0078,52149.0078,80820545404,2,5.8214,0.0
2021-02-18,414,52140.9727,52474.1055,51015.7656,51679.7969,51679.7969,52054723579,3,-0.9038,0.0
2021-02-19,415,51675.9805,56113.6523,50937.2773,55888.1328,55888.1328,63495496918,4,7.8285,0.0
2021-02-20,416,55887.3359,57505.2266,54626.5586,56099.5195,56099.5195,68145460026,5,0.3775,0.0
2021-02-21,417,56068.5664,58330.5703,55672.6094,57539.9453,57539.9453,51897585191,6,2.5352,0.0


In [24]:
# 由于假设都是全仓操作所以signal＝keep，即1代表买入持有，0代表卖出空仓
test_kl['keep'] = test_kl['signal']
# 将keep列中的nan使用向下填充的方式填充，结果使keep可以代表最终的交易持股状态
test_kl['keep'].fillna(method='ffill', inplace=True)

In [25]:
# shift(1)及np.log下面会有内容详细讲解
test_kl['benchmark_profit'] = \
    np.log(test_kl['close'] / test_kl['close'].shift(1))

# 仅仅为了说明np.log的意义，添加了benchmark_profit2，只为对比数据是否一致
test_kl['benchmark_profit2'] = \
    test_kl['close'] / test_kl['close'].shift(1) - 1

# 可视化对比两种方式计算出的profit是一致的
test_kl[['benchmark_profit', 'benchmark_profit2']].plot(subplots=True,
                                                        grid=True,
                                                        figsize=(
                                                        14, 7));

In [26]:
test_kl['trend_profit'] = test_kl['keep'] * test_kl['benchmark_profit']
test_kl['trend_profit'].plot(figsize=(14, 7))

<Axes: xlabel='date'>

In [27]:
test_kl[['benchmark_profit', 'trend_profit']].cumsum().plot(grid=True,
                                                            figsize=(
                                                            14, 7));

In [28]:
test_kl[['benchmark_profit', 'trend_profit']].cumsum().apply(
    np.exp).plot(grid=True);

### 7.1.3 趋势跟踪策略

In [37]:
# 当天收盘价格超过N1天内最高价格作为买入信号 
N1 = 42
# 当天收盘价格超过N2天内最低价格作为卖出信号
N2 = 21

In [34]:
from core.pd_helper import pd_rolling_max
# rolling_max示例序列
demo_list = np.array([1, 2, 1, 1, 100, 1000])
# 对示例序列以3个为一组，寻找每一组中的最大值
# pd.rolling_max(demo_list, window=3)
pd_rolling_max(demo_list, window=3)

array([  nan,   nan,    2.,    2.,  100., 1000.])

In [35]:
# expanding_max示例序列
demo_list = np.array([1, 2, 1, 1, 100, 1000])
from core.pd_helper  import pd_expanding_max
# pd.expanding_max(demo_list)
pd_expanding_max(demo_list)

array([   1.,    2.,    2.,    2.,  100., 1000.])

In [39]:
# 通过rolling_max方法计算最近N1个交易日的最高价
# kl_pd['n1_high'] = pd.rolling_max(kl_pd['high'], window=N1)
btc_df['n1_high'] = pd_rolling_max(btc_df['high'], window=N1)
# 表7-4所示
btc_df[0:5]

,key,open,high,low,close,adj close,volume,date_week,p_change,n1_high
date,,,,,,,,,,
2020-01-01,0,7194.8921,7254.3306,7174.9443,7200.1743,7200.1743,18565664997,2,NaN,NaN
2020-01-02,1,7202.5513,7212.1553,6935.2700,6985.4702,6985.4702,20802083465,3,-3.0273,NaN
2020-01-03,2,6984.4287,7413.7153,6914.9961,7344.8843,7344.8843,28111481032,4,5.0172,NaN
2020-01-04,3,7345.3755,7427.3857,7309.5142,7410.6567,7410.6567,18444271275,5,0.8915,NaN
2020-01-05,4,7410.4517,7544.4971,7400.5356,7411.3174,7411.3174,19725074095,6,0.0089,NaN


In [40]:
# expanding_max
# expan_max = pd.expanding_max(kl_pd['close']) 
expan_max = pd_expanding_max(btc_df['close']) 
# fillna使用序列对应的expan_max
btc_df['n1_high'].fillna(value=expan_max, inplace=True)
# 表7-5所示
btc_df[0:5]

,key,open,high,low,close,adj close,volume,date_week,p_change,n1_high
date,,,,,,,,,,
2020-01-01,0,7194.8921,7254.3306,7174.9443,7200.1743,7200.1743,18565664997,2,NaN,7200.1743
2020-01-02,1,7202.5513,7212.1553,6935.2700,6985.4702,6985.4702,20802083465,3,-3.0273,7200.1743
2020-01-03,2,6984.4287,7413.7153,6914.9961,7344.8843,7344.8843,28111481032,4,5.0172,7344.8843
2020-01-04,3,7345.3755,7427.3857,7309.5142,7410.6567,7410.6567,18444271275,5,0.8915,7410.6567
2020-01-05,4,7410.4517,7544.4971,7400.5356,7411.3174,7411.3174,19725074095,6,0.0089,7411.3174


In [41]:
from core.pd_helper import pd_rolling_min, pd_expanding_min
# 通过rolling_min方法计算最近N2个交易日的最低价格
# rolling_min与rolling_max类似
# kl_pd['n2_low'] = pd.rolling_min(kl_pd['low'], window=N2)
btc_df['n2_low'] = pd_rolling_min(btc_df['low'], window=N2)
# expanding_min与expanding_max类似
# expan_min = pd.expanding_min(kl_pd['close'])
expan_min = pd_expanding_min(btc_df['close'])
# fillna使用序列对应的eexpan_min
btc_df['n2_low'].fillna(value=expan_min, inplace=True)

In [43]:
# 当天收盘价格超过N天内的最高价或最低价, 超过最高价格作为买入信号买入股票持有 
buy_index = btc_df[btc_df['close'] > btc_df['n1_high'].shift(1)].index
btc_df.loc[buy_index, 'signal'] = 1

# 当天收盘价格超过N天内的最高价或最低价, 超过最低价格作为卖出信号
sell_index = btc_df[btc_df['close'] < btc_df['n2_low'].shift(1)].index
btc_df.loc[sell_index, 'signal'] = 0

In [44]:
btc_df.signal.value_counts().plot(kind='pie', figsize=(5, 5));

In [45]:
"""
    将信号操作序列移动一个单位，代表第二天再将操作信号执行，转换得到持股状态
    这里不shift(1)也可以，代表信号产生当天执行，但是由于收盘价格是在收盘后
    才确定的，计算突破使用了收盘价格，所以使用shift(1)更接近真实情况
"""
btc_df['keep'] = btc_df['signal'].shift(1)
btc_df['keep'].fillna(method='ffill', inplace=True)

# 计算基准收益
btc_df['benchmark_profit'] = np.log(
    btc_df['close'] / btc_df['close'].shift(1))

# 计算使用趋势突破策略的收益
btc_df['trend_profit'] = btc_df['keep'] * btc_df['benchmark_profit']

# 可视化收益的情况对比
btc_df[['benchmark_profit', 'trend_profit']].cumsum().plot(grid=True,
                                                          figsize=(
                                                          14, 7));

## 7.2 仓位控制管理

### 7.2.1 凯利公式

In [46]:
# 第一阶段走势涵盖股票上市后前100天走势情况
trade_day = 100
# 这个股票第一阶段走势函数gen_stock_price_array
def gen_stock_price_array():
    # 股票的初始价格是1元钱，即初始化100个初始价格是1元钱的np array
    price_array = np.ones(trade_day)

    # 以时间驱动100个交易日，生成100个交易日走势
    for ind in np.arange(0, trade_day - 1):
        if ind == 0:
            # 第一个交易日50%的概率结果是win
            win = np.random.binomial(1, 0.5)
        else:
            # 非第一个交易日它的涨跌与只与前一天的涨跌相关,如果前一天是上涨的
            # 那么它今天仍然是涨，如果它前一天是下跌的，那它今天就是跌
            win = price_array[ind] > price_array[ind - 1]

        if win:
            # 每次上涨只能上涨5%
            price_array[ind + 1] = (1 + 0.05) * price_array[ind]
        else:
            # 每次下跌只能下跌5%
            price_array[ind + 1] = (1 - 0.05) * price_array[ind]
    return price_array

_, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
# 运行两次，生成两种走势
price_array1 = gen_stock_price_array()
price_array1_ex = gen_stock_price_array()
# 图7-13 左图
axs[0].plot(price_array1)
# 图7-13 右图
axs[1].plot(price_array1_ex)

### 第一阶段走势函数gen_stock_price_array, 不同点只有第一个交易日100%的概率结果是win
#### np.random.binomial(1, 1)

In [47]:
# 这个股票第一阶段走势函数gen_stock_price_array, 不同点只有第一个交易日100%的概率结果是win
def gen_stock_price_array():
    # 股票的初始价格是1元钱，即初始化100个初始价格是1元钱的np array
    price_array = np.ones(trade_day)

    # 以时间驱动100个交易日，生成100个交易日走势
    for ind in np.arange(0, trade_day - 1):
        if ind == 0:
            # 第一个交易日100%的概率结果是win
            win = np.random.binomial(1, 1)
        else:
            # 非第一个交易日它的涨跌与只与前一天的涨跌相关,如果前一天是上涨的
            # 那么它今天仍然是涨，如果它前一天是下跌的，那它今天就是跌
            win = price_array[ind] > price_array[ind - 1]

        if win:
            # 每次上涨只能上涨5%
            price_array[ind + 1] = (1 + 0.05) * price_array[ind]
        else:
            # 每次下跌只能下跌5%
            price_array[ind + 1] = (1 - 0.05) * price_array[ind]
    return price_array

price_array1 = gen_stock_price_array()
plt.plot(price_array1)

In [48]:
# 第二阶段走势共覆盖了252个交易日，即一年的走势
trade_day = 252


# 这个股票第二阶段走势函数gen_stock_price_array2
def gen_stock_price_array2():
    # np.concatenate连结之前100天的走势和新的252天走势
    # np.ones(trade_day) * price_array1[-1]：即新的走势使用上一阶段走势最后
    # 一天的价格初始化这个252个交易日的新序列
    price_array = np.concatenate(
        (price_array1, np.ones(trade_day) * price_array1[-1]), axis=0)

    # concatenate操作之后：price_array有352个元素
    # len(price_array1) - 1：即ind 99开始时间驱动生成第二阶段的252个交易日
    for ind in np.arange(len(price_array1) - 1, len(price_array) - 1):
        # 获取当前交易日为基准的四个交易日数据
        last4 = price_array[ind - 3:ind + 1]
        if len(last4) == 4 and last4[-1] > last4[-2] \
                and last4[-2] > last4[-3] and last4[-3] > last4[-4]:
            # 连续上涨3天, 第四及之后天下跌的概率为55%
            win = np.random.binomial(1, 0.45)
        elif len(last4) == 4 and last4[-1] < last4[-2] \
                and last4[-2] < last4[-3] and last4[-3] < last4[-4]:
            # 连续下跌3天, 第四及之后天上涨的概率为80%
            win = np.random.binomial(1, 0.8)
        else:
            # 涨跌与只与前一天的涨跌相关，如果前一天是上涨的，
            # 那么它今天仍然是涨，如果它前一天是下跌的，那它今天就是跌
            win = price_array[ind] > price_array[ind - 1]

        if win:
            # 每次上涨只能上涨5%
            price_array[ind + 1] = (1 + 0.05) * price_array[ind]
        else:
            # 每次下跌只能下跌5%
            price_array[ind + 1] = (1 - 0.05) * price_array[ind]
    return price_array

In [49]:
import itertools
# 生成9个子画布 3*3
_, axs = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))
# 将 3 * 3转换成一个线性list
axs_list = list(itertools.chain.from_iterable(axs))
for ax in axs_list:
    # 使用gen_stock_price_array2生成9组不同的股票走势图,使用子画布绘制
    ax.plot(gen_stock_price_array2())


In [50]:
price_array2 = gen_stock_price_array2()
plt.plot(price_array2)

In [51]:
# 这个股票第三阶段走势函数gen_stock_price_array3
def gen_stock_price_array3():
    # np.concatenate连结之前352天的走势和新的交易日走势
    # np.ones(trade_day) * price_array2[-1]：即新的走势使用上一阶段走势最后
    # 一天的价格初始化len(trade_day)个交易日的新序列
    price_array = np.concatenate(
        (price_array2, np.ones(trade_day) * price_array2[-1]), axis=0)

    # concatenate操作之后：price_array352+len(trade_day)个元素
    # len(price_array2) - 1：即从ind 351开始时间驱动生成第三阶段的交易日数据
    for ind in np.arange(len(price_array2) - 1, len(price_array) - 1):
        # 获取当前交易日为基准的四个交易日数据
        last4 = price_array[ind - 3:ind + 1]
        if len(last4) == 4 and last4[-1] >= last4[-2] \
                and last4[-2] >= last4[-3] and last4[-3] >= last4[-4]:
            # 连续上涨3天, 第四及之后天下跌的概率为55%
            win = np.random.binomial(1, 0.45)
        elif len(last4) == 4 and last4[-1] < last4[-2] \
                and last4[-2] < last4[-3] and last4[-3] < last4[-4]:

            # 连续下跌3天, 第四及之后上涨的概率为80%
            win = np.random.binomial(1, 0.8)
            if not win:
                # 发生了灾难性的股价下跌，股价下跌50%
                price_array[ind + 1] = (1 - 0.50) * price_array[ind]
                # 直接continue了
                continue
        else:
            # 涨跌与只与前一天的涨跌相关，如果前一天是上涨的
            # 那么它今天仍然是涨，如果它前一天是下跌的，那它今天就是跌
            win = price_array[ind] >= price_array[ind - 1]

        if win:
            # 每次上涨只能上涨5%
            price_array[ind + 1] = (1 + 0.05) * price_array[ind]
        else:
            # 每次下跌只能下跌5%
            price_array[ind + 1] = (1 - 0.05) * price_array[ind]

        # 股价小于0.1元股价归0，即退市
        if price_array[ind + 1] <= 0.1:
            price_array[ind + 1:] = 0
            # 退市
            break

    return price_array

In [52]:
price_array3 = gen_stock_price_array3()
plt.plot(price_array3)